In [ ]:
# Setup library
## install -r requirements.txt
import os

import pandas as pd
import numpy as np

import tensorflow as tf
tf.random.set_seed(99)

import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
### Global variables
# Setup scripts (or notebook)
TFRECORD = './dataset/tfrecord/dataset_1.tfrecord'
IMG_SHAPE = (375, 4)

In [ ]:
data = tf.data.TFRecordDataset(TFRECORD)

In [ ]:
image_feature_description = {
        'vector': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int32),
        'idx': tf.io.FixedLenFeature([], tf.int64)}
def _parse_image_function(example_proto):
    # Parse the input tf.train.Example proto using the dictionary above.
    return tf.io.parse_single_example(example_proto, image_feature_description)

In [ ]:
parsed_image_dataset = data.map(_parse_image_function)
parsed_image_dataset